In [8]:
import numpy as np
import keras
import keras.backend as K

In [287]:
class NALU(keras.layers.Layer):
    
    def __init__(self, units, **kwargs):
        self.units = units
        super(NALU, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.weight_sigmoid = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_normal', name='weight_sigmoid')
        self.weight_tanh = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_normal', name='weight_tanh')
        self.weight_gate = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_normal', name='weight_gate')
        super(NALU, self).build(input_shape)
    
    def call(self, inputs):
        w = K.sigmoid(self.weight_sigmoid) * K.tanh(self.weight_tanh)
        adds = inputs @ w
        muls = K.exp(K.log(K.abs(inputs) + K.epsilon()) @ w)
        gates = K.sigmoid(inputs @ self.weight_gate)
        return gates * adds + (1-gates) * muls
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [288]:
X = X_input = keras.layers.Input((2,))
X = NALU(1)(X)
M = keras.Model(X_input, X)
M.compile('adam', 'mse')
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_80 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
nalu_86 (NALU)               (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [294]:
def toy_data(a, b):
    return a+b+1
data_input = np.random.random(size=(10000,2))*100 + 1
data_label = toy_data(data_input[:,0], data_input[:,1])

In [295]:
M.fit(data_input, data_label, epochs=1000, callbacks=[
    #keras.callbacks.ReduceLROnPlateau('loss', patience=3, verbose=1),
    #keras.callbacks.EarlyStopping('loss')
])

Epoch 1/1000
10000/10000 [==============================] - 0s 44us/step - loss: 31311.3707
Epoch 2/1000
10000/10000 [==============================] - 0s 40us/step - loss: 86.1249
Epoch 3/1000
10000/10000 [==============================] - 0s 40us/step - loss: 86.0410
Epoch 4/1000
10000/10000 [==============================] - 0s 40us/step - loss: 85.9433
Epoch 5/1000
10000/10000 [==============================] - 0s 40us/step - loss: 85.8297
Epoch 6/1000
10000/10000 [==============================] - 0s 40us/step - loss: 85.6975
Epoch 7/1000
10000/10000 [==============================] - 0s 40us/step - loss: 85.5431
Epoch 8/1000
10000/10000 [==============================] - 0s 40us/step - loss: 85.3632
Epoch 9/1000
10000/10000 [==============================] - 0s 39us/step - loss: 85.1544
Epoch 10/1000
10000/10000 [==============================] - 0s 40us/step - loss: 84.9121
Epoch 11/1000
10000/10000 [==============================] - 1s 54us/step - loss: 84.6312
Epoch 12/1000
10

10000/10000 [==============================] - 1s 56us/step - loss: 1.0249
Epoch 93/1000
10000/10000 [==============================] - 0s 50us/step - loss: 1.0157
Epoch 94/1000
10000/10000 [==============================] - 0s 40us/step - loss: 1.0072
Epoch 95/1000
10000/10000 [==============================] - 1s 52us/step - loss: 0.9993
Epoch 96/1000
10000/10000 [==============================] - 1s 50us/step - loss: 0.9937
Epoch 97/1000
10000/10000 [==============================] - 0s 50us/step - loss: 0.9876
Epoch 98/1000
10000/10000 [==============================] - 0s 41us/step - loss: 0.9831
Epoch 99/1000
10000/10000 [==============================] - 1s 50us/step - loss: 0.9780
Epoch 100/1000
10000/10000 [==============================] - 1s 51us/step - loss: 0.9738
Epoch 101/1000
10000/10000 [==============================] - 1s 51us/step - loss: 0.9704
Epoch 102/1000
10000/10000 [==============================] - 1s 51us/step - loss: 0.9672
Epoch 103/1000
10000/10000 [====

KeyboardInterrupt: 

In [296]:
M.predict(np.array([[3,2]]))

array([[5.2539997]], dtype=float32)